In [1]:
import copy
import time
import math

import numpy as np

from config.Config import Config
from exploration.explorer import Explorer
from agent.agent_handler import AgentHandler
from grid_world_generator.grid_world import GridWorld
from region_assignment.k_mean_clustring import KMeanClustring
from utils.util_functions import get_cost_matrix, stateCoordsToName
from utils.graph import get_closest_vertex_coords_on_graph_from_pos
from region_assignment.hungarian_region_assignment import HungarianRegionAssignment
from occupancy_grid_generator.occupancy_grid_generator import OccupancyGridGenerator

In [2]:
def occupancy_grid_generator():
    
    occupancy_grid = OccupancyGridGenerator()
    occupancy_grid.generate_occupancy_grid()

    # occupancy_grid.show_occupancy_grid_without_obs()
    # occupancy_grid.show_occupancy_grid_with_obs()
    
    temp_occupancy_grid_without_obs = occupancy_grid.get_occupancy_grid_without_obs()
    temp_occupancy_grid_with_obs = occupancy_grid.get_occupancy_grid_with_obs()
    
    return temp_occupancy_grid_without_obs, temp_occupancy_grid_with_obs


def grid_world(temp_occupancy_grid_without_obs):
    
    X_DIM = int(Config.GRID_WIDTH/Config.EDGE_COST)
    Y_DIM = int(Config.GRID_LEN/Config.EDGE_COST)
    print("Edge Cost:", Config.EDGE_COST, ", Width:", X_DIM, ", Height:", \
          Y_DIM, ", Sensor Range:", Config.SENSOR_RANGE, "\n")

    graph_list = []

    for i in range(Config.NO_OF_AGENTS):

        graph_list.append(GridWorld(X_DIM, Y_DIM, temp_occupancy_grid_without_obs))

        graph_list[i].run()
        # temp_graph = copy.copy(graph.get_graph())
        # graph_list[i].show_nodes_on_occupancy_grid()
        # graph_list[i].show_nodes_and_edges_with_obs_on_occupancy_grid()
        # graph_list[i].show_nodes_and_all_traversable_edges()

    temp_grid_with_nodes = graph_list[-1].get_occupancy_grid_with_nodes()
    # temp_grid_with_nodes_and_edges_with_obs = graph_list[-1].get_occupancy_grid_with_nodes_and_edges_with_obs()
    # temp_grid_with_nodes_and_all_traversable_edges = graph_list[-1].get_occupancy_grid_with_nodes_and_all_traversable_edges()
    # print(graph.graph['x0y0'])
    temp_graph_list = copy.copy(graph_list)
    
    return temp_graph_list, temp_grid_with_nodes


def k_mean_clustring(temp_occupancy_grid_without_obs):
    
    regions = KMeanClustring(temp_occupancy_grid_without_obs)

    regions.find_regions()

    # temp_regions_xy_points = regions.get_regions_xy_points()
    temp_region_centroids = regions.get_centroids()
    temp_color_map = regions.get_color_map()
    temp_grid_with_regions = copy.copy(regions.get_grid_with_regions())
    
    # regions.show_regions_with_centroids()
    # regions.show_regions()

    # ind = 0
    # for el in temp_regions_xy_points:
    #     print("Region", ind, "indices:", el)
    #     ind += 1
    print("Region centroids:\n\n", temp_region_centroids, "\n")
    print("Region color map:\n\n", temp_color_map, "\n")
    
    return temp_region_centroids, temp_color_map, temp_grid_with_regions


def region_assignment(temp_region_centroids, agenthandler, temp_grid_with_regions, temp_graph_list, temp_grid_with_nodes, temp_color_map):

    color_map = [[]] * Config.NO_OF_AGENTS
    goal_pos = [''] * Config.NO_OF_AGENTS

    region_centroids = copy.copy(temp_region_centroids)

    cost_matrix = get_cost_matrix(Config.NO_OF_AGENTS, agenthandler, region_centroids)

    hungaian_region_assignment = HungarianRegionAssignment(cost_matrix, temp_grid_with_regions)

    hungaian_region_assignment.find_regions()
    temp_regions_rows, temp_regions_cols = hungaian_region_assignment.get_regions()

    ind = 0
    
    for el in temp_regions_cols:
        temp_x, temp_y = temp_region_centroids[el]
        color_map[ind] = temp_color_map[el]
        temp_goal_pos = get_closest_vertex_coords_on_graph_from_pos(\
                            temp_graph_list[ind].get_graph(), temp_x, temp_y, Config.EDGE_COST)
        # print(temp_goal_pos)
        goal_pos[ind] = stateCoordsToName(temp_goal_pos[1], temp_goal_pos[0], Config.EDGE_COST)

        ind += 1
        
    print("\nNew Color Map Order:", color_map, "\n")
    # hungaian_region_assignment.show_assigned_regions(agenthandler, temp_region_centroids)
    # hungaian_region_assignment.show_assigned_regions(agenthandler, temp_region_centroids, temp_grid_with_nodes)

    print("Region centroids:\n")
    ind = 1
    for el in region_centroids:
        print("\tRegion:", ind, ", x:", el[0], ", y:", el[1])
        ind += 1

    print("\nAgent Positions:\n")

    for i in range(Config.NO_OF_AGENTS):
        temp_agent_pos = agenthandler.get_pos_of_agent(i)
        print("\tAgent:", i+1, ", x:", temp_agent_pos['x'], ", y:", temp_agent_pos['y'])

    print("\nCost Matrix:\n\n", np.array(cost_matrix, dtype=np.int), "\n")

    ind = 1
    for el in temp_regions_cols:
        print("Region Assigned to Agent: {} is Region {}".format(ind, el))
        ind += 1

    print("\nTotal cost:", int(hungaian_region_assignment.get_total_cost()), "\n")
    
    return goal_pos, color_map

In [3]:
def main():
    
    # ------------------ OCCUPANCY GRID GENERATOR ------------------ #
    
    temp_occupancy_grid_without_obs, temp_occupancy_grid_with_obs = occupancy_grid_generator()
    
    # -------------------------- GRID WORLD ------------------------ #
    
    temp_graph_list, temp_grid_with_nodes = grid_world(temp_occupancy_grid_without_obs)

    # ---------------------- K-MEAN-CLUSTRING ---------------------- #
    
    temp_region_centroids, temp_color_map, temp_grid_with_regions = k_mean_clustring(\
                                                                             temp_occupancy_grid_without_obs)
    
    # ------------------------ AGENT HANDLER ----------------------- #
    
    agenthandler = AgentHandler(temp_graph_list)
    
    # --------------------- REGION ASSIGNMENT ---------------------- #
    
    goal_pos, color_map_new = region_assignment(temp_region_centroids, agenthandler, temp_grid_with_regions, \
                      temp_graph_list, temp_grid_with_nodes, temp_color_map)
    
    # ------------------------- Explorer --------------------------- #
    
    explorer = Explorer(global_grid=temp_occupancy_grid_with_obs, known_grid=temp_occupancy_grid_without_obs, \
                        assigned_region_node_names=goal_pos, graph_list=temp_graph_list, \
                        agenthandler=agenthandler, color_map=color_map_new, \
                        grid_with_regions=temp_grid_with_regions, verbose=True)
    explorer.run()
    # -------------------------------------------------------------- #
    

main()

Edge Cost: 40 , Width: 25 , Height: 15 , Sensor Range: 45 

Region centroids:

 [[510 306]
 [237 305]
 [785 305]] 

Region color map:

 [[6, 106, 47], [77, 87, 244], [112, 220, 82]] 


New Color Map Order: [[112, 220, 82], [6, 106, 47], [77, 87, 244]] 

Region centroids:

	Region: 1 , x: 510 , y: 306
	Region: 2 , x: 237 , y: 305
	Region: 3 , x: 785 , y: 305

Agent Positions:

	Agent: 1 , x: 920 , y: 520
	Agent: 2 , x: 800 , y: 40
	Agent: 3 , x: 200 , y: 80

Cost Matrix:

 [[462 716 253]
 [393 622 265]
 [383 228 626]] 

Region Assigned to Agent: 1 is Region 2
Region Assigned to Agent: 2 is Region 0
Region Assigned to Agent: 3 is Region 1

Total cost: 875 

Explorer: START NODE NAMES:

['x12y22', 'x0y19', 'x1y4'] 

Explorer: ASSIGNED REGION NODE NAMES:

['x7y19', 'x7y12', 'x7y5'] 

Explorer: Number of Graphs in list: 3 

Explorer: Node Name: x12y22 , Node Position: [520, 920].
Explorer: Node Name: x0y19 , Node Position: [40, 800].
Explorer: Node Name: x1y4 , Node Position: [80, 200].

Ex